# Interactive CSV Query System


Aim : To create a query system for the Antiretroviral shipment details for the specific countries


# #Query Pipeline over Pandas DataFrames


1) Installing necessary libraries

In [ ]:
!pip install llama-index==0.9.45

2) Importing necessary libraries

In [ ]:
from llama_index.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)
from llama_index.query_engine.pandas import PandasInstructionParser
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate


3) This section used to securely prompt for and store OpenAI API key in environment variable.



In [ ]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


4) Read the CSV file "shipmentdata.csv" into a pandas DataFrame named 'df'


In [ ]:
import pandas as pd

df = pd.read_csv("/content/shipmentdata.csv")
df

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,...,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.00,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.00,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.80,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.00,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10319,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",No,See DN-4307 (ID#:83920),See DN-4307 (ID#:83920),705.79
10320,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,...,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,No,See DN-4313 (ID#:83921),See DN-4313 (ID#:83921),161.71
10321,86821,110-ZM-T30,FPQ-14784,SO-49600,DN-4316,Zambia,PMO - US,From RDC,N/A - From RDC,Truck,...,30,514526,5140114.74,9.99,0.33,Cipla Ltd A-42 MIDC Mahar. IN,No,Weight Captured Separately,Freight Included in Commodity Cost,5284.04
10322,86822,200-ZW-T30,FPQ-16523,SO-51680,DN-4334,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,17465,113871.80,6.52,0.11,Mylan (formerly Matrix) Nashik,Yes,1392,Freight Included in Commodity Cost,134.03


5) Define Modules
Here we define the set of modules:

1.Pandas prompt to infer pandas instructions from user query

2.Pandas output parser to execute pandas instructions on dataframe, get back dataframe

3.Response synthesis prompt to synthesize a final response given the dataframe
LLM

The pandas output parser specifically is designed to safely execute Python code. It includes a lot of safety checks that may be annoying to write from scratch. This includes only importing from a set of approved modules (e.g. no modules that would alter the file system like os), and also making sure that no private/dunder methods are being called.

In [ ]:
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)
pandas_output_parser = PandasInstructionParser(df)
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)
llm = OpenAI(model="gpt-3.5-turbo")

6) Build Query Pipeline
Looks like this: input query_str -> pandas_prompt -> llm1 -> pandas_output_parser -> response_synthesis_prompt -> llm2

Additional connections to response_synthesis_prompt: llm1 -> pandas_instructions, and pandas_output_parser -> pandas_output.

In [ ]:
qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=False,
)
qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
qp.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)
# add link from response synthesis prompt to llm2
qp.add_link("response_synthesis_prompt", "llm2")

7) Run Query

In [ ]:
response = qp.run(
    query_str="What are the countries managed_by pmo_us ?",
)

In [ ]:
print(response.message.content)

The countries managed by PMO - US include Côte d'Ivoire, Vietnam, Zimbabwe, Zambia, and others.


In [ ]:
response = qp.run(
    query_str="What is vendor for brand reveal ?",
)

In [ ]:
print(response.message.content)

The vendor for the brand "reveal" could not be found in the query results.


In [ ]:
response = qp.run(
    query_str="What are the shipment modes managed by pmo_us ?",
)

In [ ]:
print(response.message.content)

The shipment modes managed by PMO - US include Air, Truck, Air Charter, and Ocean.


In [ ]:
response = qp.run(
    query_str="What is the shipment mode for id 2157?",
)

In [ ]:
print(response.message.content)

The shipment mode for ID 2157 is Truck.


In [ ]:
response = qp.run(
    query_str="How many shipments are done till now?",
)

In [ ]:
print(response.message.content)

There have been 7030 shipments done till now.


In [ ]:
response = qp.run(
    query_str="How many rows and columns present in dataframe?",
)

In [ ]:
print(response.message.content)

The dataframe contains 10324 rows and 33 columns.


In [ ]:
response = qp.run(
    query_str="when is the purchase order sent vendor date to id 1 ?",
)

In [ ]:
print(response.message.content)

The purchase order with the ID 1 was sent to the vendor on SCMS-4.


In [ ]:
response = qp.run(
    query_str="when is the scheduled delivery date and delivery recoded date for id 44 ?",
)

In [ ]:
print(response.message.content)

The scheduled delivery date and delivery recorded date for ID 44 is January 8, 2007.


In [ ]:
response = qp.run(
    query_str="what is the item description for brand reveal?",
)

In [ ]:
print(response.message.content)

The item description for brand Reveal is the HIV, Reveal G3 Rapid HIV-1 Antibody Test with 30 tests per pack.


In [ ]:
response = qp.run(
    query_str="what is the item description and for brand reveal, pack price and unit price?",
)

In [ ]:
print(response.message.content)

The item description for the brand Reveal is the HIV, Reveal G3 Rapid HIV-1 Antibody Test with 30 Tests. The pack price for this item is $29.00 and the unit price is $0.97.


In [ ]:
response = qp.run(
    query_str="when is the purchase order sent to vendor date for brand reveal ?",
)

In [ ]:
print(response.message.content)

The purchase order for the brand reveal is typically sent to the vendor on the date specified in the system. Unfortunately, there was an error retrieving the exact date from the database.


In [ ]:
response = qp.run(
    query_str="How many purchase order sent to vendor date for brand reveal are captured?",
)

In [ ]:
print(response.message.content)

There were a total of 346 purchase orders sent to vendors for the brand reveal captured in the data.


In [ ]:
response = qp.run(
    query_str="what is the scheduled delivery date , delivered to client date and delivery recorded date for brand reveal ?",
)

In [ ]:
print(response.message.content)

There is no information available in the dataset for the scheduled delivery date, delivered to client date, and delivery recorded date for the brand reveal product group.


In [ ]:
print(response.message.content)

The number of vendors for the item "HIV, Reveal G3 Rapid HIV-1 Antibody Test, 30 Tests" could not be determined due to an error in running the Python code.


8) Benchmarking


9) Using Cross Encoder from Transformers :
We calculated the  score by comparing the actual answer and generated answer



In [ ]:
!pip install sentence-transformers
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
pairs=[["Côte d'Ivoire, Vietnam, Zimbabwe, Zambia","The countries managed by PMO - US include Côte d'Ivoire, Vietnam, Zimbabwe, Zambia, and others."],
 ["RANBAXY Fine Chemicals LTD , MEDMIRA EAST AFRICA LTD.","The vendor for the brand Reveal is RANBAXY Fine Chemicals LTD. and MEDMIRA EAST AFRICA LTD."],
  ["Air, Truck, Air Charter,  Ocean.","The shipment modes managed by PMO - US include Air, Truck, Air Charter, and Ocean."],
  ["Truck","The shipment mode for id 2157 is Truck."],["Date Not Captured","The purchase order sent to vendor date for ID 1 is not captured in the data."],
  ["08-01-2007, 08-01-2007","The scheduled delivery date and delivery recorded date for ID 44 are both on January 8th, 2007"],
  ["HIV, Reveal G3 Rapid HIV-1 Antibody Test, 30 Tests","The item description for brand Reveal is the Reveal G3 Rapid HIV-1 Antibody Test, which comes in a pack of 30 tests"]  ,
  ["HIV, Reveal G3 Rapid HIV-1 Antibody Test, 30 Tests,29,0.97","The item description for the Reveal brand is the HIV, Reveal G3 Rapid HIV-1 Antibody Test with 30 tests per pack. The pack price is $29 and the unit price is $0.97."],
  ["Date Not Captured,Date Not Captured,1/18/2012","The purchase order sent to the vendor date for the brand reveal was not captured in the data for most entries, but for one entry it was on 1/18/2012"],
  ["2-Jun-06 , 2-Jun-06 ,2-Jun-06,8-Aug-06, 8-Aug-06,8-Aug-06.6-Feb-12,6-Feb-12,6-Feb-12","The scheduled delivery date, delivered to client date, and delivery recorded date for the brand Reveal are as follows:1. Scheduled delivery date: 2-Jun-06, Delivered to client date: 2-Jun-06, Delivery recorded date: 2-Jun-06,2. Scheduled delivery date: 8-Aug-06, Delivered to client date: 8-Aug-06, Delivery recorded date: 8-Aug-06,3. Scheduled delivery date: 6-Feb-12, Delivered to client date: 6-Feb-12, Delivery recorded date: 6-Feb-12"]]

In [ ]:
scores = cross_encoder.predict(pairs)

In [ ]:
scores

array([ 6.830568 ,  9.202995 ,  7.631928 ,  1.0353602,  6.100611 ,
       -2.560357 ,  8.563869 ,  8.016855 ,  6.2121315,  5.0004883],
      dtype=float32)

10) Using Rouge Score :
We calculated the Rouge score by comparing the actual answer and generated answer


In [ ]:
!pip install rouge

In [ ]:
df1=pd.read_csv("/content/result_df.csv")

11) This script evaluates ROUGE scores for generated answers in a DataFrame.

 Reads a DataFrame containing 'answer' and 'Generated answer' columns from "/content/result_df.csv".
 Initializes a Rouge object for calculating text similarity scores.
 Iterates through each row, extracts answer texts, calculates ROUGE scores, and stores them in a list.
 Creates a new DataFrame for the calculated scores.
 Concatenates the original DataFrame with the ROUGE score DataFrame.
 Prints the final DataFrame containing both original data and ROUGE scores.
"""

In [ ]:
import pandas as pd
from rouge import Rouge

df1=pd.read_csv("/content/result_df.csv")

rouge = Rouge()

scores_list = []
for index, row in df1.iterrows():
    actual_answer = row['answer']
    generated_answer = row['Generated answer']

    scores = rouge.get_scores(generated_answer, actual_answer)

    scores_list.append(scores[0])

rouge_df = pd.DataFrame(scores_list)

result_df = pd.concat([df1, rouge_df], axis=1)

print(result_df)


                                            question  \
0         What are the countries managed_by pmo_us ?   
1                  What is vendor for brand reveal ?   
2    What are the shipment modes managed by pmo_us ?   
3             What is the shipment mode for id 2157?   
4  when is the purchase order sent vendor date to...   
5  when is the scheduled delivery date and delive...   
6     what is the item description for brand reveal?   
7  What is the item description and for brand rev...   
8  When is the purchase order sent to vendor date...   
9  What is the scheduled delivery date , delivere...   

                                              answer  \
0          Côte d'Ivoire, Vietnam, Zimbabwe, Zambia,   
1  RANBAXY Fine Chemicals LTD , MEDMIRA EAST AFRI...   
2                    Air, Truck, Air Charter, Ocean.   
3                                              Truck   
4                                  Date Not Captured   
5                             08-01-2007, 08-01

In [ ]:
result_df.head()

,question,answer,Generated answer,rouge-1,rouge-2,rouge-l,rouge-1.1,rouge-2.1,rouge-l.1,rouge-1,rouge-2,rouge-l
0,What are the countries managed_by pmo_us ?,"Côte d'Ivoire, Vietnam, Zimbabwe, Zambia,",The countries managed by PMO - US include Côte...,"{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499...","{'r': 1.0, 'p': 0.2857142857142857, 'f': 0.444...","{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499...","{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499...","{'r': 1.0, 'p': 0.2857142857142857, 'f': 0.444...","{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499...","{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499...","{'r': 1.0, 'p': 0.2857142857142857, 'f': 0.444...","{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499..."
1,What is vendor for brand reveal ?,"RANBAXY Fine Chemicals LTD , MEDMIRA EAST AFRI...",The vendor for the brand Reveal is RANBAXY Fin...,"{'r': 0.875, 'p': 0.4666666666666667, 'f': 0.6...","{'r': 0.75, 'p': 0.4, 'f': 0.5217391258979208}","{'r': 0.875, 'p': 0.4666666666666667, 'f': 0.6...","{'r': 0.875, 'p': 0.4666666666666667, 'f': 0.6...","{'r': 0.75, 'p': 0.4, 'f': 0.5217391258979208}","{'r': 0.875, 'p': 0.4666666666666667, 'f': 0.6...","{'r': 0.875, 'p': 0.4666666666666667, 'f': 0.6...","{'r': 0.75, 'p': 0.4, 'f': 0.5217391258979208}","{'r': 0.875, 'p': 0.4666666666666667, 'f': 0.6..."
2,What are the shipment modes managed by pmo_us ?,"Air, Truck, Air Charter, Ocean.",The shipment modes managed by PMO - US include...,"{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499...","{'r': 0.75, 'p': 0.21428571428571427, 'f': 0.3...","{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499...","{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499...","{'r': 0.75, 'p': 0.21428571428571427, 'f': 0.3...","{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499...","{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499...","{'r': 0.75, 'p': 0.21428571428571427, 'f': 0.3...","{'r': 1.0, 'p': 0.3333333333333333, 'f': 0.499..."
3,What is the shipment mode for id 2157?,Truck,The shipment mode for id 2157 is Truck.,"{'r': 1.0, 'p': 0.125, 'f': 0.2222222202469136}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 1.0, 'p': 0.125, 'f': 0.2222222202469136}","{'r': 1.0, 'p': 0.125, 'f': 0.2222222202469136}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 1.0, 'p': 0.125, 'f': 0.2222222202469136}","{'r': 1.0, 'p': 0.125, 'f': 0.2222222202469136}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 1.0, 'p': 0.125, 'f': 0.2222222202469136}"
4,when is the purchase order sent vendor date to...,Date Not Captured,The purchase order sent to vendor date for ID ...,"{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}"


12) Saved the final DataFrame with ROUGE scores to 'result_df.csv', excluding the index column.


In [ ]:
result_df.to_csv('result_df.csv', index=False)